<a href="https://colab.research.google.com/github/ianjamesbarnett/2022NFLBigDataBowl/blob/main/nextposFFNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import io
import torchvision as tv 
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pickle
import gc





1.   frame picture
2.   (x0,y0,a0,s0,o0) and (x1,y1,a1,s1,o1)
3.   

For a given frame you need



1) 
For a given player, get their current (x0,y0) and get their next (x1,y1,a1,s1,o1)
2) extract the subimage around (x0,y0).
3) Run the CNN with the subimage as input and (x1,y1,a1,s1,o1) as output.
4) Use the CNN to construct next images

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
data_dir = "/content/drive/MyDrive/NFLBigDataBowl2022"
os.chdir(data_dir)

#YEAR="2018";maxplays=756
YEAR="2019";maxplays=749
#YEAR="2020";maxplays=855


filein=open(data_dir+"/tracking"+YEAR+".csv",'r')
line=filein.readline()

filein2=open(data_dir+"/games.csv",'r')
filein2.readline()
game_d={}
for line in filein2:
    line_v=[x.strip('\"') for x in line.strip("\n").split(",")]
    game_d[line_v[0]]=line_v[1:]

filein3=open(data_dir+"/plays.csv",'r')
filein3.readline()
play_d={}
for line in filein3:
    line_v=[x.strip('\"') for x in line.strip("\n").split(",")]
    play_d[line_v[0]+"-"+line_v[1]]=line_v[2:]


In [ ]:
# gets the frame of all received punts
#pret={}
#for line in filein:
#    line_v=line.split(",")
#    if line_v[8].strip("\"")=="punt_received":
#        gpID = line_v[15]+"-"+line_v[16]
#        if gpID not in pret:
#            pret[gpID]={}
#        pret[gpID][line_v[9]]=line_v[1:8]+line_v[11:14]


In [ ]:
def PlotFieldPosition(temp):
    plotdata = {'x':[],'y':[],'s':[],'a':[],'o':[],'dir':[],'jerseyNumber':[],'position':[],'team':[],'color':[]}
    for key in temp:
        plotdata['x'].append(float(temp[key][0]))
        plotdata['y'].append(float(temp[key][1]))
        plotdata['s'].append(float(temp[key][2]))
        plotdata['a'].append(float(temp[key][3]))
        plotdata['o'].append(temp[key][5])
        plotdata['dir'].append(temp[key][6])
        plotdata['jerseyNumber'].append(temp[key][7])
        plotdata['position'].append(temp[key][8])
        plotdata['team'].append(temp[key][9])
        if temp[key][9].strip("\"")=="football":
            plotdata['color'].append(0)    
        elif temp[key][9].strip("\"")=="away":
            plotdata['color'].append(1)    
        else:
            plotdata['color'].append(2)    
    plt.scatter('y','x',c='color',data=plotdata)
    plt.show()

#PlotFieldPosition(pret[key1])  
#PlotFieldPosition(pret[key2])  





# get IDs of all plays to turn into videos (returned punts and kicks)
catchframe = {}
pID_s=set()
filein.seek(0)
for line in filein:
    line_v=line.split(",")
    if line_v[8].strip("\"")=="punt_received" or line_v[8].strip("\"")=="kick_received":
        gpID = line_v[15]+"-"+line_v[16]
        if play_d[gpID][6]=="Return":
            pID_s.add(gpID)
            catchframe[gpID]=line_v[14]


        
# create video files
ret_vids = {} # 3 layers of dictionaries: playID, frameID, playerID
filein.seek(0)
for line in filein:
    line_v=line.split(",")
    gpID = line_v[15]+"-"+line_v[16]
    if gpID not in pID_s:
        continue
    if gpID not in ret_vids:
        ret_vids[gpID]={}
    if line_v[14] not in ret_vids[gpID]:
        ret_vids[gpID][line_v[14]]={}
    ret_vids[gpID][line_v[14]][line_v[9]]=line_v[1:8]+line_v[11:14]+[line_v[17].strip("\n")]

# Create multi-channel images
#x,y,s,a,dis,o,dir,jersey,pos,team,playdirectino
pIDs=list(ret_vids.keys())


In [ ]:
#radius=200
#plID=pIDs[0]
#player='NA'
#frID=list(ret_vids[plID].keys())[-1]
#frID='1'

def GetClosePlayers(player,plID,frID,radius=250):
  curd=ret_vids[plID][frID]
  ofIDs=list(curd.keys())
  cx0=float(curd[player][0])
  cy0=float(curd[player][1])
  distv=[]
  xrelv=[]
  yrelv=[]
  idv=[]
  for id in ofIDs:
    if id=='NA':
      continue
    xrel=float(curd[id][0])-cx0
    yrel=float(curd[id][1])-cy0
    curdist=np.sqrt(xrel**2+yrel**2)
    if curdist>radius:
      continue
    distv.append(curdist)
    idv.append(id)
    xrelv.append(xrel)
    yrelv.append(yrel)
  distv=np.array(distv)
  yrelv=np.array(yrelv)
  xrelv=np.array(xrelv)
  idv=np.array(idv)
  sortperm=np.argsort(distv)
  #distv[np.argsort(distv)]
  #idv[np.argsort]
  idv=idv[sortperm]
  distv=distv[sortperm]
  yrelv=yrelv[sortperm]
  xrelv=xrelv[sortperm]
  return({'idv':idv,'distv':distv,'xrelv':xrelv,'yrelv':yrelv})

#GetClosePlayers(player,plID,frID,2)






'DEN'

In [ ]:
def os2xyspeed(o,s):
  if o<90:
    ot=o*2*np.pi/360
    x=s*np.sin(ot)
    y=s*np.cos(ot)
  elif o<180:
    ot=(180-o)*2*np.pi/360
    x=s*np.sin(ot)
    y=-s*np.cos(ot)
  elif o<180:
    ot=(o-180)*2*np.pi/360
    x=-s*np.sin(ot)
    y=-s*np.cos(ot)
  else:
    ot=(360-o)*2*np.pi/360
    x=-s*np.sin(ot)
    y=s*np.cos(ot)
  return([x,y])


def xyspeed2os(xsp,ysp):
  s=np.sqrt(xsp**2+ysp**2)
  ot=180*np.arccos(abs(ysp)/s)/np.pi
  if xsp>=0 and ysp>=0:
    o=ot
  elif xsp>=0 and ysp<0:
    o=180-ot
  elif xsp<0 and ysp<0:
    o=ot+180
  else:
    o=360-ot
  return([s,o])
  
xyspeed2os(1,-1)

[1.4142135623730951, 135.0]

In [ ]:
#x,y,s,a,dis,o,dir,jersey,pos,team,playdirectino
y=[]
x=[]
for i in range(len(pIDs)): # for each play
  if len(y)>1000000:
    print(i)
    break
  plID=pIDs[i]
  teamkick=play_d[plID][4]
  homeACR=game_d[plID.split("-")[0]][4]
  awayACR=game_d[plID.split("-")[0]][5]
  if teamkick==homeACR:
    homekick=True
  else:
    homekick=False
  frIDs=list(ret_vids[plID].keys())
  for j in range(len(frIDs)-1): # for each frame in a play except the last
    frID=frIDs[j]
    if float(catchframe[plID])<float(frID): # after catch
      continue
    playerIDs=list(ret_vids[plID][frID].keys())
    
    for k in range(len(playerIDs)):
      playerID=playerIDs[k]
      if playerID=="NA":
        continue
      # get outcome vector: deltax,deltay, a, xspd, yspd
      cx=float(ret_vids[plID][frID][playerID][0])
      cy=float(ret_vids[plID][frID][playerID][1])
      cs=float(ret_vids[plID][frID][playerID][2])
      ca=float(ret_vids[plID][frID][playerID][3])
      cdis=float(ret_vids[plID][frID][playerID][4])
      co=float(ret_vids[plID][frID][playerID][5])
      nx=float(ret_vids[plID][frIDs[j+1]][playerID][0])
      ny=float(ret_vids[plID][frIDs[j+1]][playerID][1])
      ns=float(ret_vids[plID][frIDs[j+1]][playerID][2])
      na=float(ret_vids[plID][frIDs[j+1]][playerID][3])
      ndis=float(ret_vids[plID][frIDs[j+1]][playerID][4])
      no=float(ret_vids[plID][frIDs[j+1]][playerID][5])
      temp=os2xyspeed(no,ns)
      nxspd=temp[0]
      nyspd=temp[1]
      temp=os2xyspeed(co,cs)
      cxspd=temp[0]
      cyspd=temp[1]
      cury=[nx-cx,ny-cy,na,nxspd,nyspd]
      ### get 3 closest players
      gcpout=GetClosePlayers(playerID,plID,frID,radius=250)
      # get predictor vector: a, xspd, yspd, dis----plus--- top 3 closest reldis, relx, rely, a, xspd, yspd
      curx=[ca,cxspd,cyspd,cdis]
      for l in range(5):
        ID2=gcpout['idv'][l]
        cx2=float(ret_vids[plID][frID][ID2][0])
        cy2=float(ret_vids[plID][frID][ID2][1])
        cs2=float(ret_vids[plID][frID][ID2][2])
        ca2=float(ret_vids[plID][frID][ID2][3])
        cdis2=float(ret_vids[plID][frID][ID2][4])
        co2=float(ret_vids[plID][frID][ID2][5])
        temp=os2xyspeed(co2,cs2)
        cxspd2=temp[0]
        cyspd2=temp[1]
        if ret_vids[plID][frID][ID2][9]==ret_vids[plID][frID][playerID][9]:
          sameteam=1
        else:
          sameteam=0
        curx+=[np.sqrt((cx2-cx)**2+(cy2-cy)**2),cx2-cx,cy2-cy,ca2,cxspd2,cyspd2,sameteam]
      # also add the relx and rely of the ball to the player
      cx2=float(ret_vids[plID][frID]['NA'][0])
      cy2=float(ret_vids[plID][frID]['NA'][1])
      cretind=0
      if (homekick and ret_vids[plID][frID][playerID][9].strip("\"")=="home") or ((not homekick) and ret_vids[plID][frID][playerID][9].strip("\"")=="away"):
        cretind=1
      curx+=[cx2,cy2,cretind]
      y.append(cury)
      x.append(curx)


      



In [ ]:
def Get_Triangle_Coords(cx,cy,a,s,o):
    minh=1;maxh=3
    minw=.66;maxw=1.33
    maxs=30
    s=np.min((maxs,s))
    h=minh+(np.log(1+s/maxs)/np.log(2))*(maxh-minh)
    maxa=30
    a=np.min((maxa,a))
    w=minw+(np.log(1+a/maxa)/np.log(2))*(maxw-minw)
    b=np.arctan(w/(2*h))
    if o<90:
        x1=(h/np.cos(b)-w*np.sin(2*np.pi*(90-o)/360)/(2*np.sin(b+2*np.pi*o/360)))*np.sin(b+2*np.pi*o/360)
        y1=np.sqrt(np.abs(h**2-x1**2))
        x2=(w/2)*np.sin(2*np.pi*(90-o)/360)
        y2=-(w/2)*np.cos(2*np.pi*(90-o)/360)
        x3=-x2
        y3=-y2
    elif o<180:
        op = 180-o
        xp1=(h/np.cos(b)-w*np.sin(2*np.pi*(90-op)/360)/(2*np.sin(b+2*np.pi*op/360)))*np.sin(b+2*np.pi*op/360)
        yp1=np.sqrt(np.abs(h**2-xp1**2))
        x1=xp1
        y1=-yp1
        x2p=(w/2)*np.sin(2*np.pi*(90-op)/360)
        y2p=-(w/2)*np.cos(2*np.pi*(90-op)/360)
        x3p=-x2p
        y3p=-y2p
        x2=x2p
        y2=-y2p
        x3=x3p
        y3=-y3p
    elif o<270:
        op = o-180
        xp1=(h/np.cos(b)-w*np.sin(2*np.pi*(90-op)/360)/(2*np.sin(b+2*np.pi*op/360)))*np.sin(b+2*np.pi*op/360)
        yp1=np.sqrt(np.abs(h**2-xp1**2))
        x1=-xp1
        y1=-yp1
        x2p=(w/2)*np.sin(2*np.pi*(90-op)/360)
        y2p=-(w/2)*np.cos(2*np.pi*(90-op)/360)
        x3p=-x2p
        y3p=-y2p
        x2=-x2p
        y2=-y2p
        x3=-x3p
        y3=-y3p
    else:
        op = 360-o
        xp1=(h/np.cos(b)-w*np.sin(2*np.pi*(90-op)/360)/(2*np.sin(b+2*np.pi*op/360)))*np.sin(b+2*np.pi*op/360)
        yp1=np.sqrt(np.abs(h**2-xp1**2))
        x1=-xp1
        y1=yp1
        x2p=(w/2)*np.sin(2*np.pi*(90-op)/360)
        y2p=-(w/2)*np.cos(2*np.pi*(90-op)/360)
        x3p=-x2p
        y3p=-y2p
        x2=-x2p
        y2=y2p
        x3=-x3p
        y3=y3p
    return(([cx+x1,cx+x2,cx+x3],[cy+y1,cy+y2,cy+y3]))
        

# do log of acceleration+1.5


def fig2img(fig,dpi=20):
    """Convert a Matplotlib figure to a PIL Image and return it"""
    buf = io.BytesIO()
    fig.savefig(buf,bbox_inches='tight',pad_inches=0,dpi=dpi)
    buf.seek(0)
    img = Image.open(buf)
    return img

def GenRGBFromPlayFrame(plID,frID,dpi=30):
    teamkick=play_d[plID][4]
    homeACR=game_d[plID.split("-")[0]][4]
    awayACR=game_d[plID.split("-")[0]][5]
    outd={} # this will store 3 channels, home team position, away team position, and football.
    resparam=.5
    curFrInfo=ret_vids[plID][frID]
    for channel in range(3):
        fig=plt.figure(figsize=(120*resparam,53.3*resparam))
        plt.axis('off')
        plt.xlim([0,120])
        plt.ylim([0,53.3])
        for ele in curFrInfo:
            xval=float(curFrInfo[ele][0])
            yval=float(curFrInfo[ele][1])
            if ele=='NA':
                if channel==2:
                    plt.scatter([xval],[yval],c="green",s=500)
                    continue
                continue
            cs=float(curFrInfo[ele][2])
            ca=float(curFrInfo[ele][3])
            co=float(curFrInfo[ele][5])
            pval=Get_Triangle_Coords(xval,yval,ca,cs,co)
            if curFrInfo[ele][9]=='\"home\"' and teamkick==homeACR and channel==0:
                col="red"
                plt.fill(pval[0],pval[1],c=col)
            elif curFrInfo[ele][9]=='\"home\"' and teamkick==awayACR and channel==1:
                col="blue"
                plt.fill(pval[0],pval[1],c=col)
            elif curFrInfo[ele][9]=='\"away\"' and teamkick==homeACR and channel==1:
                col="blue"
                plt.fill(pval[0],pval[1],c=col)
            elif curFrInfo[ele][9]=='\"away\"' and teamkick==awayACR and channel==0:
                col="red"
                plt.fill(pval[0],pval[1],c=col)
        img = fig2img(fig,dpi=dpi)
        convert_tensor = tv.transforms.ToTensor()
        to_grayscale = tv.transforms.Grayscale()
        imgt=convert_tensor(to_grayscale(img))
        if curFrInfo['NA'][10].strip("\"") == 'left':
            imgt[0]=torch.fliplr(imgt[0])
        epsilon=.01
        imgt[0]=1-torch.floor(imgt[0]+epsilon)
        outd[channel]=imgt[0]
        plt.close()
    imgout=torch.zeros([3,imgt.size()[1],imgt.size()[2]])
    for channel in range(3):
        imgout[channel]=outd[channel]
    return(imgout)
        

# Get outcome variable for a given play
# If the outcome is yards gained then this is a function of the frame as well.
def GetOutcomeVar(plID,frID):
    #subtract yards from catch to current position from total yards
    # NA:condition on no penalty
    if play_d[plID][15] != 'NA':
        return('NA')
    ENDYARDLINE=float(ret_vids[plID][str(len(ret_vids[plID]))]['NA'][0])
    CURYARDLINE=float(ret_vids[plID][frID]['NA'][0])
    if ret_vids[plID]['1']['NA'][10].strip("\"")=='right':
        return(CURYARDLINE-ENDYARDLINE)
    else:
        return(ENDYARDLINE-CURYARDLINE)


In [ ]:
def testerrv(net,testloader):
  outerr=[]
  with torch.no_grad():
        for data in testloader:
            images, labels = data
            # calculate outputs by running images through the network
            images=images.float()
            labels=labels.float()
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            #print(outputs,labels)
            runtot=np.zeros(5)
            denom=float(labels.shape[0])
            for i in range(int(denom)):
              for j in range(5):
                runtot[j]+=abs(labels[i,j]-outputs[i,j])/denom
            outerr.append(runtot)
  totouterr=np.zeros(5)
  for i in range(len(outerr)):
    for j in range(5):
      totouterr[j]+=outerr[i][j]/len(outerr)
  print(outputs[0])
  print(labels[0])
  return(totouterr)

In [ ]:
yt=torch.tensor(y)
xt=torch.tensor(x)

In [ ]:
xt

tensor([[ 3.1000e-01, -1.0267e-01,  1.2271e-01,  ...,  4.5190e+01,
          2.6320e+01,  0.0000e+00],
        [ 1.9000e-01, -6.3529e-02, -2.9395e-02,  ...,  4.5190e+01,
          2.6320e+01,  0.0000e+00],
        [ 4.7000e-01, -6.8000e-02, -1.6614e-02,  ...,  4.5190e+01,
          2.6320e+01,  0.0000e+00],
        ...,
        [ 7.7000e-01,  8.5757e+00,  2.7100e-01,  ...,  1.1044e+02,
          3.2770e+01,  1.0000e+00],
        [ 2.4400e+00,  5.4435e-01, -1.6421e+00,  ...,  1.1044e+02,
          3.2770e+01,  0.0000e+00],
        [ 1.6200e+00,  4.5704e-01,  3.8328e+00,  ...,  1.1044e+02,
          3.2770e+01,  0.0000e+00]], dtype=torch.float64)

In [ ]:

trainset=torch.utils.data.TensorDataset(xt[:int(len(yt)*.9)],yt[:int(len(yt)*.9)])
testset=torch.utils.data.TensorDataset(xt[int(len(yt)*.9):],yt[int(len(yt)*.9):])

batch_size=64
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,shuffle=True, num_workers=0)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,shuffle=True, num_workers=0)




#in_d=pickle.load(open("vid3ch"+YEAR+".pickle",'rb'))
#xt=in_d['xt']
#yt=in_d['yt']

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(42, 50)
        self.fc2 = nn.Linear(50,25)
        self.fc3 = nn.Linear(25,10)
        self.fc4 = nn.Linear(10, 5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)#.squeeze()
        return x
    
    

net = Net()


In [ ]:
#modelpath=data_dir+"/nextposFFNN"+"619"+".pt"
#net.load_state_dict(torch.load(modelpath))

<All keys matched successfully>

In [ ]:
criterion = nn.MSELoss(reduction="mean")
optimizer = optim.Adam(net.parameters(), lr=0.001)#, momentum=0.95)
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.95)
testcor=[]
traincor=[]
for epoch in range(2000):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs=inputs.float()
        labels=labels.float()
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        #print(i)
        if i % 10000 == 9999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10000))
            running_loss = 0.0
#    curtraincor=tensorcorr(trainloader,net)
#    curtestcor=tensorcorr(testloader,net)
#    print("testcor: ", curtestcor, "; traincor: ",curtraincor)
#    traincor.append(curtraincor)
#    testcor.append(curtestcor)
#    torch.save(net.state_dict(), modelpath)
#    print(testerrv(net,trainloader))
    testerr=testerrv(net,testloader)
    print(testerr,sum(testerr))
    if epoch % 20==19:
      torch.save(net.state_dict(),data_dir+"/nextposFFNN_take2_"+str(epoch)+".pt")
print('Finished Training')


[1, 10000] loss: 0.063
tensor([0.3411, 0.0470, 2.4409, 3.4589, 0.1732])
tensor([ 0.3400, -0.0100,  2.4800,  3.3383,  0.1061])
[0.0703325  0.07208639 0.19489999 0.20017725 0.33853534] 0.8760314652970463
[2, 10000] loss: 0.061
tensor([-0.0829, -0.0365,  3.6594, -1.1188, -2.1597])
tensor([-0.2400,  0.0100,  3.3700, -1.6924, -1.9738])
[0.06813896 0.07083302 0.1861996  0.18825945 0.25511589] 0.7685469077583226
[3, 10000] loss: 0.060
tensor([ 0.3677, -0.6436,  3.9490,  1.4970, -6.7536])
tensor([ 0.5800, -0.3500,  4.3300,  1.2023, -6.4184])
[0.06208509 0.07033769 0.19027604 0.20536376 0.26211006] 0.7901726451842359
[4, 10000] loss: 0.059
tensor([-0.5723,  0.0793,  0.8982, -5.8789, -1.6008])
tensor([-0.6200, -0.0500,  0.6800, -6.0523, -1.5596])
[0.06209481 0.06722781 0.18493286 0.20514537 0.26126773] 0.7806685734213443
[5, 10000] loss: 0.058
tensor([ 0.0218, -0.0258,  0.5544,  0.2615, -0.1065])
tensor([0.0300, 0.0100, 0.5800, 0.3695, 0.0196])
[0.06680537 0.07329469 0.18236113 0.18507743 0.2717

KeyboardInterrupt: ignored

In [ ]:
print(outputs[1])
print(labels[1])


tensor([-0.0072,  0.0406,  0.1818,  0.1057,  0.0422],
       grad_fn=<SelectBackward0>)
tensor([0.0100, 0.0000, 0.1700, 0.1056, 0.0308])


In [ ]:
testerrv(net,testloader)

tensor([ 0.0058,  0.0040,  0.1958, -0.0716,  0.0089])
tensor([-0.0100,  0.0000,  0.0300, -0.0210,  0.0454])


array([0.05644637, 0.06513546, 0.17668964, 0.18321152, 0.24541968])